# Arrange Documents according to section name
- Read one patient(a file)'s all records 
- Separate all documents within a patient file
- After dividing the documents, sort them by date
- Separately remove the AP part of SOAP from the sorted document and insert a separator (THIS_IS_AP)
- The remaining parts are pasted sequencely.
- for example, a document can be arrange like this:
<pre>
[START_AP]
Assessment & Plan : su -> januvia
MTF up
3 m
[END_AP]
History <- 소견 : FBS 123
Hb A 1 c 4 . 5
LDL Chol . 25
Physical examination <- 소견 : 75 k /175 cm
[END_DOC]
</pre>

## Section types

In [ ]:
section_types_paths = ["./data/02_type_to_file_links/sections_visits_2011to2020_task2.txt"]

section_types = {}
for i in range(len(section_types_paths)):
    section_types_path = section_types_paths[i]
    file = open(section_types_path, "r")
    lines = file.readlines()
    for l in range(len(lines)):
        if lines[l]=="\n":
            continue
        doctype = lines[l].split("\t")[0].strip("\n")
        section = lines[l].split("\t")[1].strip("\n")    
        key = doctype+"\t"+section

        val = lines[l].split("\t")[2].strip("\n")
        section_types[key] = val

    file.close()

## Heldouts

In [ ]:
heldout_path = "../preprocessing/01_data4finetune/"

train_patients = [
    "pts_SNUH_visit_2011to2020_heldout_train.txt",
]

test_patients = [
    "pts_SNUH_visit_2011to2020_heldout_test.txt",
]


# heldouts
def get_heldouts(heldout_array_files):
    heldouts = []
    for h in range(len(heldout_array_files)):
        file = open(heldout_path +"/"+ heldout_array_files[h])
        lines = file.readlines()
        heldouts_cate = []
        for l in range(len(lines)):
            pts = lines[l].split("/")[-1]
            pts = pts.split(".")[0]
    #         print(str(pts))
            heldouts_cate.append(pts)
        heldouts.append(heldouts_cate)
    return heldouts

train_pts = get_heldouts(train_patients)
test_pts = get_heldouts(test_patients)

print("len(train_pts[0]): ", len(train_pts[0]))
print("len(test_pts[0]): ", len(test_pts[0]))

## Arrange Documents SOAP

In [ ]:
import os

In [ ]:
def arrange_soap(target_path, section_types):
    target_patient = target_path.split("/")[-1].split(".txt")[0]
    target_group = target_path.split("/")[-2]
    target_category = target_path.split("/")[-3]

    file = open(target_path, "r")
    lines = file.readlines()
    file.close()
    
    
    date_last = ""
    doctype_last = ""
    section_last = ""

    all_documents = []
    
    document_so = []
    document_ap = []
    
    for l in range(len(lines)):
        line = lines[l].strip("\n")

        if line=="":
            continue
        
        line = line.split("\t")
        date = line[0]
        doctype = line[1]
        section = line[2]
        content = line[3]
        
        key = doctype+"\t"+section
        is_AP = section_types[key]
        if is_AP=="증상 및 검사":
            mode = "SO"
        else:
            mode = "AP"

        # When looping through the same document or for the very first time
        if (date==date_last and doctype==doctype_last) or (l==0):
            """the same"""
        # When entering another document, move the existing document and empty it before adding it.
        else:
            """different"""
            # output a document
            if len(document_ap)!=0 or len(document_so)!=0:
                all_documents.append("[START_AP]")
                if len(document_ap)>0:
                    all_documents.append("\n".join(document_ap))
                all_documents.append("[END_AP]")
                if len(document_so)>0:
                    all_documents.append("\n".join(document_so))
                all_documents.append("[END_DOC]"+"\n")
            document_ap = []
            document_so = []
            
            
            # Sections are also reset when the document changes
            section_last=""

        # Continue same section
        if section_last == section:
            #document.append(content)
            if mode=="AP":
                document_ap.append(content)
            else:
                document_so.append(content)
            
        # When other sections appear
        else:
            if mode=="AP":
                document_ap.append(section + " : " + content)
            else:
                document_so.append(section + " : " + content)            
            
            
        date_last = date
        doctype_last = doctype
        section_last = section

        
    # output a document
    if len(document_ap)!=0 or len(document_so)!=0:
        all_documents.append("[START_AP]")
        if len(document_ap)>0:
            all_documents.append("\n".join(document_ap))
        all_documents.append("[END_AP]")
        if len(document_so)>0:
            all_documents.append("\n".join(document_so))
        all_documents.append("[END_DOC]"+"\n")
    document_ap = []
    document_so = []
                
    
    output_directory = "./data/03_docsplit/"+str(target_category)
    #print("output_directory: ", output_directory)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    output_directory = "./data/03_docsplit/"+str(target_category)+"/"+str(target_group)
    #print("output_directory: ", output_directory)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    outtext = "\n".join(all_documents)
    file = open(output_directory+"/"+str(target_patient)+".txt", "w")
    file.write(outtext)
    file.close()    
    

In [ ]:
import glob
import os

docperfile = 50

categories = [
    "visits_2011to2020",
]

# train
for c in range(len(categories)):
    category = categories[c]
    
    for t in range(len(train_pts[c])):
        target_patient = train_pts[c][t]
        print("target_patient: ", train_pts[c][t])
        
        zeropoint = 0
        for p in range(len(target_patient)):
            #print(target_patient[p])
            if target_patient[p]!="0":
                zeropoint = p
                break
        
        pt_id = int(target_patient[zeropoint:])
        #print("pt_id: ", pt_id)
        groupname = str(((pt_id//docperfile)+1)*docperfile)       
        
        target_path = "./data/"+str(category)+"/"+str(groupname)+"/"+str(target_patient)+".txt"
        print("target_path :", target_path)
        
        arrange_soap(target_path, section_types)
        

print('Done trainset')


In [ ]:

# test
for c in range(len(categories)):
    category = categories[c]
    
    for t in range(len(test_pts[c])):
        target_patient = test_pts[c][t]
        print("target_patient: ", train_pts[c][t])
        
        zeropoint = 0
        for p in range(len(target_patient)):
            #print(target_patient[p])
            if target_patient[p]!="0":
                zeropoint = p
                break
        
        pt_id = int(target_patient[zeropoint:])
        #print("pt_id: ", pt_id)
        groupname = str(((pt_id//docperfile)+1)*docperfile)       
        
        target_path = "./data/"+str(category)+"/"+str(groupname)+"/"+str(target_patient)+".txt"
        print("target_path :", target_path)
        
        arrange_soap(target_path, section_types)
        

print('Done testset')
